In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re

In [3]:
input_dir = "/home/daniel/Documents/VGP/results"
taxonomy_file = "/home/daniel/Documents/VGP/ID_Species.tsv"
taxonomy_df = pd.read_csv(taxonomy_file, sep="\t")
taxonomy_dict = dict(zip(taxonomy_df["Assembly_ID"], taxonomy_df["Taxon_Group"]))
data = []

In [4]:
#Anterior total
for file in os.listdir(input_dir):
    if file.endswith(".rmstats"):  
        current_class = None
        with open(os.path.join(input_dir, file)) as f:
            for line in f:
                line = line.strip()
                if line.startswith("Class:"):
                    current_class = line.split("Class:")[1].strip()
                elif current_class and line and not line.startswith('|') and not line.startswith('Total-') and not line.startswith('-'):
                    match = re.match(r'^(\S+)\s+(\d+)\s+([\d\.]+)\s+([\d\.]+)%\s+([\d\.]+)%$', line)
                    if match:
                        family = match.group(1)
                        number_of_elements = int(match.group(2))
                        size_of_family = float(match.group(3))
                        percentage_class = float(match.group(4))
                        percentage_total = float(match.group(5))
                        data.append([file, current_class, family, number_of_elements, size_of_family, percentage_class, percentage_total])

In [4]:
#Por familias
for file in os.listdir(input_dir):
    if file.endswith(".rmstats"):  
        current_class = None
        file_path = os.path.join(input_dir, file)
        with open(file_path) as f:
            for line in f:
                line = line.strip()
                if line.startswith("Class:"):
                    current_class = line.split("Class:")[1].strip()
                elif current_class and line and not line.startswith('|') and not line.startswith('Total-') and not line.startswith('-'):
                    match = re.match(r'^(\S+)\s+(\d+)\s+([\d\.]+)\s+([\d\.]+)%\s+([\d\.]+)%$', line)
                    if match:
                        family = match.group(1)
                        number_of_elements = int(match.group(2))
                        size_of_family = float(match.group(3))
                        percentage_class = float(match.group(4))
                        percentage_total = float(match.group(5))
                        assembly_id = file.replace(".rmstats", "")
                        taxon = taxonomy_dict.get(assembly_id, "Unknown")
                        data.append([assembly_id, taxon, current_class, family, number_of_elements, size_of_family, percentage_class, percentage_total])

In [5]:
#Anterior
df = pd.DataFrame(data, columns=["File", "Class", "Family", "Number_of_Elements", "Size_of_Family", "Percentage_of_Class", "Percentage_of_Total_Size"])
print(df.head()) 

                      File          Class         Family  Number_of_Elements  \
0  GCA_020745775.1.rmstats       ARTEFACT       ARTEFACT                  12   
1  GCA_020745775.1.rmstats  Simple_repeat  Simple_repeat              236735   
2  GCA_020745775.1.rmstats           tRNA           tRNA                3462   
3  GCA_020745775.1.rmstats           rRNA           rRNA                 175   
4  GCA_020745775.1.rmstats           DNA?           DNA?                3158   

   Size_of_Family  Percentage_of_Class  Percentage_of_Total_Size  
0           693.0                100.0                      0.00  
1      10385343.0                100.0                     12.95  
2        478288.0                100.0                      0.60  
3         20548.0                100.0                      0.03  
4        634293.0                100.0                      0.79  


In [5]:
#Actual
df = pd.DataFrame(data, columns=["Assembly", "Taxon_Group", "Class", "Family", "Number_of_Elements", "Size_of_Family", "Percentage_of_Class", "Percentage_of_Total_Size"])
families = df["Family"].unique()

In [15]:
def plot_items(variable):
    for fam in sorted(families):
        df_fam = df[df["Family"] == fam]
        if df_fam.empty:
            continue  # evita errores si no hay datos para esta familia

        class_name = df_fam["Class"].iloc[0]  # asumimos que todas las filas tienen la misma clase

        plt.figure(figsize=(10, 6))
        sns.boxplot(data=df_fam, x="Taxon_Group", y=variable)
        plt.title(f"Boxplot - Class: {class_name} | Family: {fam} ({variable.replace('_', ' ')})")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f"boxplot_{class_name}_{fam}_{variable}.png")
        plt.close()

In [17]:
plot_items("Percentage_of_Total_Size")

In [18]:
plot_items("Number_of_Elements")

In [19]:
classes = df["Class"].unique()
print(classes)

['ARTEFACT' 'Simple_repeat' 'tRNA' 'rRNA' 'DNA?' 'Satellite' 'LTR' 'SINE'
 'Unspecified' 'snRNA' 'RC?' 'Low_complexity' 'RC' 'Unknown' 'DNA' 'LTR?'
 'LINE' 'scRNA' 'srpRNA' 'SINE?' 'Other' 'PLE' 'Retroposon' 'Satellite?'
 'Segmental' 'RNA']


In [20]:
def plot_items_class(variable):
    for cls in sorted(classes):
        plt.figure(figsize=(10, 6))
        sns.boxplot(data=df[df["Class"] == cls], x="Taxon_Group", y=variable)
        plt.title(f"Boxplot - {cls} ({variable.replace('_', ' ')})")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f"classboxplot_{cls}_{variable}.png")
        plt.close()

In [21]:
plot_items_class("Percentage_of_Total_Size")

In [22]:
plot_items_class("Number_of_Elements")

In [28]:
unknowns = df[df["Taxon_Group"]=="Unknown"]

In [31]:
unknowns["Assembly"].unique()

array(['GCA_035609145.1', 'GCA_040894005.1', 'GCF_963455315.1',
       'GCA_964264875.1', 'GCA_040954625.1', 'GCA_964195595.1',
       'GCF_901001135.1', 'GCA_031021105.1', 'GCF_029281585.2',
       'GCA_964212035.1', 'GCA_964146895.1', 'GCA_023637945.1',
       'GCF_929443795.1', 'GCA_044704955.1', 'GCA_028372415.1',
       'GCA_023333525.1', 'GCF_901765095.1', 'GCA_011762505.2',
       'GCA_047511675.1', 'GCA_040894355.1', 'GCA_964237555.1',
       'GCA_042242105.1', 'GCA_039877785.1', 'GCA_036169615.1',
       'GCA_963455315.2', 'GCF_902459505.1', 'GCA_030463535.1'],
      dtype=object)